In [2]:
import pandas as pd
import numpy as np

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from numpy import array

## importing libraries

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import time
from sklearn import preprocessing
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.metrics
import math
from numpy import mean
from numpy import std

from sklearn.metrics import mean_squared_error, r2_score


print('Done importing libraries')

Done importing libraries


In [3]:
df2 = pd.read_csv('data/allWinter_delhi.csv')
df2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df2.drop(["a"], axis=1, inplace=True)



Ozone = df2['O3']

OzoneP24 = Ozone.shift(-24)
OzoneP24 = OzoneP24.replace(np.nan, Ozone.mean())
df2['O3P24'] = OzoneP24

df2.info()
print(df2.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10814 entries, 0 to 10813
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        10814 non-null  int64  
 1   City         10814 non-null  object 
 2   Date         10814 non-null  object 
 3   Time         10814 non-null  object 
 4   PM2.5        10814 non-null  float64
 5   PM10         10814 non-null  float64
 6   NO           10814 non-null  float64
 7   NO2          10814 non-null  float64
 8   NOx          10814 non-null  float64
 9   NH3          10814 non-null  float64
 10  CO           10814 non-null  float64
 11  SO2          10814 non-null  float64
 12  O3           10814 non-null  float64
 13  Benzene      10814 non-null  float64
 14  Toluene      10814 non-null  float64
 15  Xylene       10814 non-null  float64
 16  O3P24        10814 non-null  float64
 17  Temperature  10814 non-null  float64
dtypes: float64(14), int64(1), object(3)
memory usa

In [4]:
## defining variables

#X = df2[['PM10', 'NO', 'NO2', 'CO', 'SO2', 'O3', 'Toluene', 'Temp']] # , 'Humid', Xylene
X = df2[['PM10', 'NO', 'NO2', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train1.shape[0]))
print("Size of testing dataset: {} rows".format(X_test1.shape[0]))

print('Done defining variables')

Size of training dataset: 9732 rows
Size of testing dataset: 1082 rows
Done defining variables


In [5]:
## scaling variables

'''importing libraries'''
from sklearn.preprocessing import RobustScaler

scaler = preprocessing.RobustScaler()
X_train_scaled1 = scaler.fit_transform(X_train1)
X_test_scaled1 = scaler.fit_transform(X_test1)

print('Done scaling :)')

Done scaling :)


In [8]:
## linear regression
t0 = time.time()

'''importing library'''
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold

'''create linear regressor object'''
lnRegressor = LinearRegression()

'''fitting regressor'''
lnRegressor.fit(X_train1, y_train)

# cv stuff

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

cvln_r2scores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvln_r2scores)
cvln_rmsescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvln_rmsescores)
cvln_maescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvln_maescores)

# calculating r^2 adj score

cvln_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvln_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvln_adj.append(adj_r2)    

cvln_r2score = round((np.mean(cvln_r2scores)), 4)
cvln_adjscore = round((np.mean(cvln_adj)), 4)
cvln_rmsescore = (round(abs(np.mean(cvln_rmsescores)),2))
cvln_maescore = (round(abs(np.mean(cvln_maescores)),2))


print(cvln_r2score, cvln_adjscore, cvln_rmsescore, cvln_maescore)

t1 = time.time()
total = t1-t0

print("done training linear regressor | time taken: %f seconds" %total)
yPredln = lnRegressor.predict(X_test1)
print('The average R2 value for linear regressor is :', cvln_r2score)

[0.50562967 0.55995601 0.5440999  0.61453085 0.57363799 0.57046787
 0.5299323  0.50465611 0.56146021 0.64897428]
[-24.88957942 -24.0572911  -24.74276775 -24.74425918 -23.49970437
 -23.76896235 -26.29057536 -27.36536002 -25.4723718  -22.69474541]
[-13.84046195 -14.57958903 -14.92540276 -14.95408572 -14.7034538
 -14.75273965 -15.13608205 -15.55122177 -15.77072726 -13.99941039]
0.5613 0.5609 24.75 14.82
done training linear regressor | time taken: 0.488455 seconds
The average R2 value for linear regressor is : 0.5613


In [9]:
## kneighbors regression
print("STARTING: KN")

t0 = time.time()

'''importing library'''
from sklearn.neighbors import KNeighborsRegressor

'''create regressor object'''
knRegressor = KNeighborsRegressor(n_neighbors = 4, metric = 'minkowski', p = 1)

'''fitting regressor'''
knRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvkn_r2scores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvkn_r2scores)
cvkn_rmsescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvkn_rmsescores)
cvkn_maescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvkn_maescores)

# calculating r^2 adj score

cvkn_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvkn_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvkn_adj.append(adj_r2)    

cvkn_r2score = round((np.mean(cvkn_r2scores)), 4)
cvkn_adjscore = round((np.mean(cvkn_adj)), 4)
cvkn_rmsescore = (round(abs(np.mean(cvkn_rmsescores)),2))
cvkn_maescore = (round(abs(np.mean(cvkn_maescores)),2))

print(cvkn_r2score, cvkn_adjscore, cvkn_rmsescore, cvkn_maescore)


t1 = time.time()
total = t1-t0

print("done training knn regressor | time taken: %f seconds" %total)

yPredkn = knRegressor.predict(X_test_scaled1)
print('The average R2 value for KNN Regressor is :', cvln_r2score)

STARTING: KN
[0.58249331 0.67387542 0.69335559 0.63846828 0.64343144 0.69199974
 0.63113654 0.66384974 0.65432816 0.75725007]
[-22.87299721 -20.71049323 -20.29228555 -23.96364329 -21.49040868
 -20.12741179 -23.28909981 -22.54314427 -22.61501521 -18.8727594 ]
[-12.2765554  -12.21501548 -11.8127025  -13.15738379 -12.66022975
 -12.12463135 -12.26245089 -13.22002275 -12.58618962 -11.23134354]
0.663 0.6627 21.68 12.35
done training knn regressor | time taken: 2.324967 seconds
The average R2 value for KNN Regressor is : 0.5613


In [11]:
## svr regressor
print("STARTING: SVR")

t0 = time.time()

'''importing library'''
from sklearn.svm import SVR

'''create regressor object'''
svrRegressor = SVR(C = 10, gamma = 0.1, kernel= 'rbf')

'''fitting regressor'''
svrRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvsvr_r2scores = cross_val_score(svrRegressor, X_train_scaled1, y_train, cv = 10, scoring = 'r2')
print(cvsvr_r2scores)
cvsvr_rmsescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvsvr_rmsescores)
cvsvr_maescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvsvr_maescores)


# calculating r^2 adj score

cvsvr_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvsvr_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvsvr_adj.append(adj_r2)

cvsvr_r2score = round((np.mean(cvsvr_r2scores)), 4)
cvsvr_adjscore = round((np.mean(cvsvr_adj)), 4)
cvsvr_rmsescore = (round(abs(np.mean(cvsvr_rmsescores)),2))
cvsvr_maescore = (round(abs(np.mean(cvsvr_maescores)),2))

print(cvsvr_r2score, cvsvr_adjscore, cvsvr_rmsescore, cvsvr_maescore)

t1 = time.time()
total = t1-t0

print("done training svr regressor | time taken: %f seconds" %total)

yPredsvr = svrRegressor.predict(X_test_scaled1)
print('The average R2 value for SVR Regressor is :', cvsvr_r2score)

STARTING: SVR
[0.58986913 0.65227395 0.66139353 0.64846091 0.66875098 0.63358854
 0.58005679 0.62375821 0.62597487 0.69843862]
[-36.77913658 -38.27893816 -37.98855819 -41.73956235 -37.84514209
 -38.16838583 -40.04535228 -40.36030895 -40.24595818 -40.2892819 ]
[-22.85729166 -25.00477437 -24.85850509 -26.07644691 -25.06894893
 -25.27845999 -25.42392797 -26.04587016 -26.16575465 -25.74694573]
0.6383 0.6379 39.17 25.25
done training svr regressor | time taken: 294.698972 seconds
The average R2 value for SVR Regressor is : 0.6383


In [12]:
## defining variables WITH MULTICOLLINEARITY: random forest, decision tree, maybe adaboost, and xgboost

X = df2[['PM10', 'NO', 'NO2', 'NOx', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train.shape[0]))
print("Size of testing dataset: {} rows".format(X_test.shape[0]))

print('Done defining NEW variables')

Size of training dataset: 9732 rows
Size of testing dataset: 1082 rows
Done defining NEW variables


In [13]:
## random forest regressor
print("STARTING: rf")
t0 = time.time()

'''importing library'''
from sklearn.ensemble import RandomForestRegressor

'''create regressor object'''
rfRegressor = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=250) 

'''fitting regressor'''
rfRegressor.fit(X_train, y_train)

# cv stuff

cvrf_r2scores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'r2') # x_train was scaled for some reason, check if it changes things?
print(cvrf_r2scores)
cvrf_rmsescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvrf_rmsescores)
cvrf_maescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvrf_maescores)


# calculating r^2 adj score

cvrf_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvrf_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvrf_adj.append(adj_r2)

cvrf_r2score = round((np.mean(cvrf_r2scores)), 4)
cvrf_adjscore = round((np.mean(cvrf_adj)), 4)
cvrf_rmsescore = (round(abs(np.mean(cvrf_rmsescores)),2))
cvrf_maescore = (round(abs(np.mean(cvrf_maescores)),2))

print(cvrf_r2score, cvrf_adjscore, cvrf_rmsescore, cvrf_maescore)

print(cvrf_r2score, cvrf_adjscore)

t1 = time.time()
total = t1-t0

print("done training random forest regressor | time taken: %f seconds" %total)
yPredrf = rfRegressor.predict(X_test)
print('The average R2 value for Random Forest Regressor is :', cvrf_r2score)

## printing feature importances

importances = rfRegressor.feature_importances_

sorted_indices = np.argsort(importances)[::-1]

print('feature importances:::')
print(sorted_indices)

STARTING: rf
[0.63467938 0.73585469 0.75948305 0.73230656 0.74621074 0.73065095
 0.72924099 0.72584982 0.74737799 0.76720131]
[-21.39579615 -18.63889358 -17.9715807  -20.62046748 -18.13049965
 -18.82218841 -19.95314453 -20.35832454 -19.3330545  -18.48187874]
[-11.10630308 -10.91065133 -10.63728174 -11.51873528 -10.82191662
 -11.30896782 -11.09247168 -11.52810376 -11.41621901 -10.43989284]
0.7309 0.7306 19.37 11.08
0.7309 0.7306
done training random forest regressor | time taken: 928.528381 seconds
The average R2 value for Random Forest Regressor is : 0.7309
feature importances:::
[ 7  8  0  3  4 10  2  1  6  5  9]


In [14]:
## decision tree regressor
print('STARTING: dt')
t0 = time.time()

'''importing library'''
from sklearn.tree import DecisionTreeRegressor

'''create regressor object'''
dtRegressor = DecisionTreeRegressor(random_state=0, max_depth = 6)

'''fitting regressor'''
dtRegressor.fit(X_train,y_train)

# cv stuff

cvdt_r2scores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvdt_r2scores)
cvdt_rmsescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvdt_rmsescores)
cvdt_maescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvdt_maescores)


# calculating r^2 adj score

cvdt_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvdt_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvdt_adj.append(adj_r2)

cvdt_r2score = round((np.mean(cvdt_r2scores)), 4)
cvdt_adjscore = round((np.mean(cvdt_adj)), 4)
cvdt_rmsescore = (round(abs(np.mean(cvdt_rmsescores)),2))
cvdt_maescore = (round(abs(np.mean(cvdt_maescores)),2))

print(cvdt_r2score, cvdt_adjscore, cvdt_rmsescore, cvdt_maescore)

t1 = time.time()
total = t1-t0

print("done training decision tree regressor | time taken: %f seconds" %total)

yPreddt = dtRegressor.predict(X_test)
print('The average R2 value for Decision Tree Regressor is :', cvdt_r2score)

STARTING: dt
[0.51787233 0.6369105  0.65781011 0.51555886 0.63722138 0.58477845
 0.61973711 0.6247086  0.64003626 0.66540641]
[-24.57946276 -21.85272085 -21.43616176 -27.73959739 -21.67674116
 -23.36965677 -23.64622726 -23.81946863 -23.07779235 -22.15718872]
[-13.03463411 -13.15190528 -12.9243185  -14.71568262 -13.26454476
 -13.95488434 -14.00986125 -13.77053144 -13.94383977 -12.98995331]
0.61 0.6096 23.34 13.58
done training decision tree regressor | time taken: 2.680516 seconds
The average R2 value for Decision Tree Regressor is : 0.61


In [15]:
## adaboost regressor
print('STARTING: adaboost regressor')
t0 = time.time()

'''importing library'''
from sklearn.ensemble import AdaBoostRegressor

'''create regressor object'''
adaRegressor = AdaBoostRegressor(random_state=0, learning_rate = 0.1, n_estimators=100)

'''fitting regressor'''
adaRegressor.fit(X_train, y_train)

# cv stuff

cvada_r2scores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvada_r2scores)
cvada_rmsescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvada_rmsescores)
cvada_maescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvada_maescores)

# calculating r^2 adj score

cvada_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvada_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvada_adj.append(adj_r2)

cvada_r2score = round((np.mean(cvada_r2scores)), 4)
cvada_adjscore = round((np.mean(cvada_adj)), 4)
cvada_rmsescore = (round(abs(np.mean(cvada_rmsescores)),2))
cvada_maescore = (round(abs(np.mean(cvada_maescores)),2))

print(cvada_r2score, cvada_adjscore, cvada_rmsescore, cvada_maescore)

t1 = time.time()
total = t1-t0

print("done training Adaboost regressor | time taken: %f seconds" %total)

yPredada = adaRegressor.predict(X_test)
print('The average R2 value for Adaboost Regressor is :', cvada_r2score)

STARTING: adaboost regressor
[0.52284557 0.58654879 0.59409086 0.58695324 0.57769878 0.5751233
 0.57883121 0.55658978 0.61093194 0.64604608]
[-24.4523634  -23.31904754 -23.34682634 -25.61410887 -23.38752786
 -23.63980282 -24.88559207 -25.89110656 -23.99261991 -22.7892068 ]
[-15.08801231 -15.763722   -15.92440845 -16.90473988 -15.6946033
 -15.69167482 -16.23721802 -17.10800562 -15.77853869 -15.36445367]
0.5836 0.5831 24.13 15.96
done training Adaboost regressor | time taken: 105.751082 seconds
The average R2 value for Adaboost Regressor is : 0.5836


In [16]:
## xgboost regressor
print('STARTING: xgboost regressor')
t0 = time.time()

'''importing library'''
import xgboost as xgb

'''create regressor object'''
xgbRegressor = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, n_estimators=300, verbosity = 0, random_state = 0, silent = True)

'''fitting regressor'''
xgbRegressor.fit(X_train, y_train)

# cv stuff

cvxgb_r2scores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvxgb_r2scores)
cvxgb_rmsescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvxgb_rmsescores)
cvxgb_maescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvxgb_maescores)


# calculating r^2 adj score

cvxgb_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvxgb_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvxgb_adj.append(adj_r2)

cvxgb_r2score = round((np.mean(cvxgb_r2scores)), 4)
cvxgb_adjscore = round((np.mean(cvxgb_adj)), 4)
cvxgb_rmsescore = (round(abs(np.mean(cvxgb_rmsescores)),2))
cvxgb_maescore = (round(abs(np.mean(cvxgb_maescores)),2))

print(cvxgb_r2score, cvxgb_adjscore, cvxgb_rmsescore, cvxgb_maescore)

t1 = time.time()
total = t1-t0

print("done training xgboost regressor | time taken: %f seconds" %total)

yPredxgb = xgbRegressor.predict(X_test)
print('The average R2 value for xgboost Regressor is :', cvxgb_r2score)

STARTING: xgboost regressor
[0.65285026 0.77288838 0.77223642 0.73861036 0.75415039 0.69971235
 0.73601061 0.73710557 0.71830861 0.77545732]
[-20.85690099 -17.28296918 -17.48862138 -20.37622989 -17.84464501
 -19.87380986 -19.70212725 -19.93601961 -20.41510759 -18.15119826]
[-10.84694021 -10.6023258  -10.50074724 -11.28669648 -10.51182582
 -11.11831481 -10.88107685 -11.19677339 -11.70176556 -10.27514772]
0.7357 0.7354 19.19 10.89
done training xgboost regressor | time taken: 1165.556929 seconds
The average R2 value for xgboost Regressor is : 0.7357


In [17]:
##### printing cross-validations scores
# printing cross-validaion r^2 scores

print('The R^2 value for Linear Regression is         :', cvln_r2score)
print('The R^2 value for KNN Regressor is             :', cvkn_r2score)
print('The R^2 value for SVM Regressor is             :', cvsvr_r2score)
print('The R^2 value for Random Forests Regressor is  :', cvrf_r2score)
print('The R^2 value for Decision Tree Regressor is   :', cvdt_r2score)
print('The R^2 value for AdaBoost Regressor is        :', cvada_r2score)
print('The R^2 value for XGBoost Regressor is         :', cvxgb_r2score)

# printing cross-validation adjusted r^2 scores

print('The Adj. R^2 value for Linear Regression is        :', cvln_adjscore)
print('The Adj. R^2 value for KNN Regressor is            :', cvkn_adjscore)
print('The Adj. R^2 value for SVM Regressor is            :', cvsvr_adjscore)
print('The Adj. R^2 value for Random Forests Regressor is :', cvrf_adjscore)
print('The Adj. R^2 value for Decision Tree Regressor is  :', cvdt_adjscore)
print('The Adj. R^2 value for AdaBoost Regressor is       :', cvada_adjscore)
print('The Adj. R^2 value for XGBoost Regressor is        :', cvxgb_adjscore)

# printing cross-validation rmse scores

print('The RSME value for Linear Regression is         :', cvln_rmsescore)
print('The RSME value for KNN Regressor is             :', cvkn_rmsescore)
print('The RSME value for SVM Regressor is             :', cvsvr_rmsescore)
print('The RSME value for Random Forests Regressor is  :', cvrf_rmsescore)
print('The RSME value for Decision Tree Regressor is   :', cvdt_rmsescore)
print('The RSME value for AdaBoost Regressor is        :', cvada_rmsescore)
print('The RSME value for XGBoost Regressor is         :', cvxgb_rmsescore)

# printing cross-validation mae scores

print('The MAE value for Linear Regression is        :', cvln_maescore)
print('The MAE value for KNN Regressor is            :', cvkn_maescore)
print('The MAE value for SVM Regressor is            :', cvsvr_maescore)
print('The MAE value for Random Forests Regressor is :', cvrf_maescore)
print('The MAE value for Decision Tree Regressor is  :', cvdt_maescore)
print('The MAE value for AdaBoost Regressor is       :', cvada_maescore)
print('The MAE value for XGBoost Regressor is        :', cvxgb_maescore)

The R^2 value for Linear Regression is         : 0.5613
The R^2 value for KNN Regressor is             : 0.663
The R^2 value for SVM Regressor is             : 0.6383
The R^2 value for Random Forests Regressor is  : 0.7309
The R^2 value for Decision Tree Regressor is   : 0.61
The R^2 value for AdaBoost Regressor is        : 0.5836
The R^2 value for XGBoost Regressor is         : 0.7357
The Adj. R^2 value for Linear Regression is        : 0.5609
The Adj. R^2 value for KNN Regressor is            : 0.6627
The Adj. R^2 value for SVM Regressor is            : 0.6379
The Adj. R^2 value for Random Forests Regressor is : 0.7306
The Adj. R^2 value for Decision Tree Regressor is  : 0.6096
The Adj. R^2 value for AdaBoost Regressor is       : 0.5831
The Adj. R^2 value for XGBoost Regressor is        : 0.7354
The RSME value for Linear Regression is         : 24.75
The RSME value for KNN Regressor is             : 21.68
The RSME value for SVM Regressor is             : 39.17
The RSME value for Rand